## Pre-trained Network used on a new set of classes


ATTRIBUTION: Sample based on "Fine-tune InceptionV3 on a new set of classes" from https://keras.io/applications/  


**Pre-requisites**:
on an OCI GPU instance NVIDIA NGC:  
nvidia-docker run -it -d -p 8888:8888 -p 6006:6006 -v /home/ubuntu:/shared nvcr.io/nvidia/tensorflow:18.04-py2

make two directory:  ''**data/training**' and '**data/test**'
to reference into:  

train_data_dir =  '**data/training**' #contains classes to train  
validation_data_dir = '**data/test**' #contains classes for validation  

Upload in training directory the images, creating a subfolder directory for each class  
Upload in test directory images for validation, creating a subfolder  directory for each class  


### Pre-requisites library

In [ ]:
! pip install keras
! pip install pillow
! pip install Flask
! pip install Flask-Uploads
! pip install Flask-Uploads --upgrade
! pip install matplotlib

In [ ]:
! pwd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Check Tensorflow release

In [ ]:
import tensorflow as tf; print(tf.__version__)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
import PIL
import os
from keras.models import load_model

### Setup & data preparation

##### Basic Parameters

In [ ]:
# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir =  'cars-dataset/training' #contains classes to train
validation_data_dir = 'cars-dataset/validation' #contains classes for validation
test_data_dir = 'cars-dataset/test'  #contains classes for testing

nb_epoch = 7
model_name = "inception_retrained_model7"
nb_batchsize = 64

##### Training & Validation Data load and augmentation

In [ ]:
# Data for Training and Validation
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        #shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)
print ("\nTraining directory:")
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_batchsize,
    class_mode='categorical'
)

label_map = (train_generator.class_indices)

print ("\nLabel map in training directory:")
print (label_map)
print ("\nValidation directory:")
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_batchsize,
    class_mode='categorical'
)

nb_train_samples = sum([len(files) for r, d, files in os.walk(train_data_dir)])
nb_validation_samples = sum([len(files) for r, d, files in os.walk(validation_data_dir)])
nb_classes  = sum([len(d) for r, d, files in os.walk(validation_data_dir)])


### Training step: go to Load model re-trained cell to skip training phase

In [ ]:
from tensorflow.python.client import device_lib

#print(device_lib.list_local_devices())
print("Get available GPUs")
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
nb_classes =  len(label_map)

# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(nb_classes, activation='softmax')(x)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
print("base_model")
#for i, layer in enumerate(model.layers):
#    print(i, layer.name)
    

In [ ]:
if os.path.isfile(model_name+".h5"):
    model = load_model(model_name+".h5")
    print("re-training pre-trained model")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 300 layers and unfreeze the rest:
#for layer in model.layers[:300]:
for layer in model.layers[:311]:
    layer.trainable = False
#for layer in model.layers[300:]:
for layer in model.layers[312:]:
    layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy',metrics=['acc'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
# model.fit_generator(...)
# fine-tune the model
if (nb_batchsize>nb_validation_samples):
    nb_validation_steps = 1
else:
    nb_validation_steps =int(nb_validation_samples/nb_batchsize)

if (nb_batchsize>nb_train_samples):
    nb_train_samples=1
else:
    nb_train_steps =int(nb_train_samples/nb_batchsize)

print (nb_train_steps)
print (nb_validation_steps)


history=model.fit_generator(
        train_generator,
        #samples_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        steps_per_epoch=nb_train_samples,
        validation_data=validation_generator,
        validation_steps=nb_validation_steps,
        verbose=1,
        shuffle=True)
        #nb_val_samples=nb_validation_samples,
        #steps_per_epoch=int(nb_train_samples/nb_batchsize)


model.save(model_name+".h5")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

history_dict = history.history
history_dict.keys()

loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['acc']
val_acc_values = history_dict['val_acc']


epochs = range(1, nb_epoch + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')          
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')     
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
plt.clf()                                    

plt.plot(epochs,acc_values, 'bo', label='Training acc')
plt.plot(epochs, val_acc_values, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
print("predictions")
print(predictions)

### Load model re-trained

In [ ]:
from keras.preprocessing import image
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
import numpy

# returns a compiled model
# identical to the previous one
model = load_model(model_name+".h5")



### Inference function

In [ ]:
import json
def inference(imagepath):
    img_path = imagepath
    img = image.load_img(img_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    preds = model.predict(x)
    
    # Decode results
    inv_map = {v: k for k, v in label_map.iteritems()}
    i=0
    a = []
    for prob in preds[0]:
        a.append([i,int(prob*100)])
        i+=1

    sorted= numpy.argsort(a,axis=0)
    final =[]
    for elem in sorted:
        final.append([inv_map[elem[1]],int(preds[0][elem[1]]*100)])
    inference= numpy.flip(final,axis=0)
    
    partJson= '{ \"name\":\"'+imagepath+'\",  \"results\": [ '
  
    for i in range(0,nb_classes):
        partJson+='{ \"class' +str(i+1) + '\":\"' + str(inference[i][0]) + '\", \"value\":\"' + str(inference[i][1]) + '\" },' 
    json = partJson[:-1]+ ']  }'
    return json

### Test on local image

In [ ]:
print("Test on a local image:")
result = inference(test_data_dir+'/door.jpg')
print (result)

## REST Server for inference

In [ ]:
import os
from flask import Flask, request, redirect, url_for,Response
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = os.getcwd()+'/img'
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

#app = Flask(__name__)
app = Flask(os.getcwd())

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def root_dir():  # pragma: no cover
    #print('TEST')
    #return os.path.abspath(os.path.dirname(__file__))
    return os.getcwd()


def get_file(filename):  # pragma: no cover
    try:

        src = os.path.join(root_dir(), filename)
        # Figure out how flask returns static files
        # Tried:
        # - render_template
        # - send_file
        # This should not be so non-obvious
        return open(src).read()
    except IOError as exc:
        return str("NOT_FOUND")

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        print(root_dir())
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit a empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            #-----------------------------------------------------------------------------
            # INFERENCE
            result = inference(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            
            #-----------------------------------------------------------------------------
            
            #file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            #open(os.path.join(app.config['UPLOAD_FOLDER'], filename+'.ck'),'w').close()
            #return redirect(url_for('upload_file',filename=filename))
            return result

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <p><input type=file name=file accept="image/*">
         <input type=submit value=Upload>
    </form>
    '''

@app.route('/inference', methods=['GET'])
def metrics():  # pragma: no cover
    fileResponse=request.args.get('filename', '')
    content = get_file(UPLOAD_FOLDER+"/"+fileResponse+".json")
    print(UPLOAD_FOLDER+"/"+fileResponse+".json")
    return Response(content, mimetype="application/json")

if __name__ == '__main__':
    print(root_dir())
    #app.run(debug=True)
    app.run(host='0.0.0.0',port=6006)